In [1]:
import pandas as pd
import numpy as np

In [10]:
dat_df = pd.read_csv("all_data.csv", sep=',')

In [11]:
## change naming 
dat_df['cue_lum_congruency'] = dat_df['cue_lum_congruency'].astype(bool)
dat_df['cue_lum_congruency'].replace({True: 'congruent', False: 'incongruent'}, inplace=True)

## change to categorical variable
#dat_df['soa'] = pd.Categorical(dat_df['soa'])

## make sure of a type
dat_df['hit'] = dat_df['hit'].astype(bool)
dat_df['miss'] = dat_df['miss'].astype(bool)
dat_df['got_reward'] = dat_df['got_reward'].astype(bool)

## performance from nrec
dat_df['performance_from_nrec'] = dat_df.iloc[:, 7:9].idxmax(axis=1).str.replace('performance_', '')
dat_df.rename(columns={'hit': 'hit_from_nrec', 'miss': 'miss_from_nrec'}, inplace=True)

## performance from IDT algorithm
dat_df['performance_from_eye'] = dat_df.apply(lambda x: 'hit' if x['lum_change_id'] == x['response_from_tracking'] else 'miss', axis=1)
dat_df['hit_from_eye'] = dat_df['performance_from_eye'].apply(lambda x: 1 if x == 'hit' else 0).astype(bool)
dat_df['miss_from_eye'] = dat_df['performance_from_eye'].apply(lambda x: 1 if x == 'miss' else 0).astype(bool)

dat_df['was_congruent'] = dat_df['cue_lum_congruency'] == 'congruent'
dat_df['was_incongruent'] = dat_df['cue_lum_congruency'] == 'incongruent'

dat_df = dat_df.dropna(subset=['response_from_tracking'])

dat_df_ = dat_df.copy()

In [12]:
dat_df = dat_df[dat_df['soa']>0]
dat_df = dat_df[dat_df['soa']<0.7]

In [13]:
main_soas = [0.02, 0.05, 0.1, 0.2, 0.3]

def closest_main_soa(x):
    if x in main_soas:
        return x
    return min(main_soas, key=lambda y: abs(x - y))

dat_df['soa'] = dat_df['soa'].apply(closest_main_soa)

In [14]:
for run_id in dat_df['run'].unique():
    for soa_ in dat_df['soa'].unique():
        if dat_df[(dat_df['run'] == run_id) & (dat_df['soa'] == soa_)].shape[0] < 20: 
            dat_df = dat_df[~((dat_df['run'] == run_id) & (dat_df['soa'] == soa_))]

dat_df_eo = dat_df[dat_df['condition'] == 'eye_only']
dat_df_eh = dat_df[dat_df['condition'] == 'eye_head']


In [15]:
dat_df_eo.to_csv("dat_df_eo.csv")

In [16]:
dat_df_eh.to_csv("dat_df_eh.csv")

In [127]:
count_dat_df_eh = dat_df_eh.dropna(subset=['saccade_time_to_lum']).groupby('soa').size().reset_index(name='n')
count_dat_df_eh[count_dat_df_eh['n'] !=0]

,soa,n
0,0.02,2349
1,0.05,5050
2,0.10,4828
3,0.20,1490
4,0.30,6532


In [128]:
count_dat_df_eo = dat_df_eo.dropna(subset=['saccade_time_to_lum']).loc[dat_df_eo['cue_amplitude'] == 'normal'].groupby('soa').size().reset_index(name='n')
count_dat_df_eo[count_dat_df_eo['n'] !=0]

,soa,n
0,0.02,8561
1,0.05,1225
2,0.10,9303
3,0.20,9251
4,0.30,16990


In [129]:
count_dat_df_eo_large = dat_df_eo.dropna(subset=['saccade_time_to_lum']).loc[dat_df_eo['cue_amplitude'] == 'large'].groupby('soa').size().reset_index(name='n')
count_dat_df_eo_large[count_dat_df_eo_large['n'] !=0]

,soa,n
0,0.05,1889
1,0.10,1931
2,0.30,2024
